In [2]:
import pandas as pd

# Fuzzy Matching - 04082020

In [3]:
new=pd.read_csv('data/2015 Booth List.csv')
old=pd.read_csv('data/2019 Booth List.csv')

In [6]:
# I. Fuzzy Matching the intra localities - both old & new

In [7]:
unique_loc=new['locality'].unique().tolist()
type(unique_loc)

list

In [7]:
from fuzzywuzzy import process, fuzz

/mnt/c/Users/gufra/OneDrive/Documents/Workspace/Jutaav/code_base/aurangabad/venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# LOCALITY FUZZY - edit 10082020 - (dont use, do direct match on booth first

In [8]:
# Intra fuzzy for NEW
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

new1=new.merge(high_score_sort,how='left',left_on=new['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

new1['loc_sort1']=np.where(new1['_merge']=='left_only',new1['locality'],new1['sorted_loc_sort'])

# Intra Fuzzy for OLD

unique_loc1=old['locality'].unique().tolist()
score_sort = [(x,) + i
             for x in unique_loc 
             for i in process.extract(x, unique_loc,     scorer=fuzz.token_sort_ratio)]


#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['loc_sort','match_sort','score_sort'])
similarity_sort.head()

#Derive representative values
import numpy as np
similarity_sort['sorted_loc_sort'] = np.minimum(similarity_sort['loc_sort'], similarity_sort['match_sort'])
similarity_sort.head()

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 90) &
                                      (similarity_sort['loc_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_loc_sort'] != similarity_sort['match_sort'])]

old1=old.merge(high_score_sort,how='left',left_on=old['locality'].str.upper().str.strip(),\
         right_on=high_score_sort['match_sort'].str.upper().str.strip(),indicator=True)

old1['loc_sort1']=np.where(old1['_merge']=='left_only',old1['locality'],old1['sorted_loc_sort'])

In [78]:
old1[old1['loc_sort1']=='Ambedkar Aawasiya High School']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
211,AMBEDKAR AAWASIYA HIGH SCHOOL,212,Ambedkar Aawasiya High School Kishanganj East ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
212,AMBEDKAR AAWASIYA HIGH SCHOOL,213,Ambedkar Aawasiya High School Kishanganj Middl...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School
213,AMBEDKAR AAWASIYA HIGH SCHOOL,214,Ambedkar Aawasiya High School Kishanganj West ...,Ambedkar Aawasiya High School,NaN,NaN,NaN,NaN,left_only,Ambedkar Aawasiya High School


In [80]:
old1[old1['loc_sort1']=='pahadakatata']

,key_0,Number,Name,locality,loc_sort,match_sort,score_sort,sorted_loc_sort,_merge,loc_sort1
128,PAHADAKATTA,129,"Panchayat Bhawan Pahadakatta, East Part",pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata
129,PAHADAKATTA,130,Panchayat Bhawan Pahadakatta West Part,pahadakatta,pahadakatata,pahadakatta,96.0,pahadakatata,both,pahadakatata


# BOOTH FUZZY - USE THIS

In [4]:
# Inter Fuzzy between new1 and old1

def checker(wrong_options,correct_options):
    
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append("100")
        else:
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [5]:
str2Match=old['booth_name'].tolist()
strOptions=new['booth_name'].tolist()

In [8]:
name_match,ratio_match=checker(str2Match,strOptions)
df1 = pd.DataFrame()
df1['old_names']=pd.Series(str2Match)
df1['correct_names']=pd.Series(name_match)
df1['correct_ratio']=pd.Series(ratio_match)
# df1.to_excel(‘matched_names.xlsx’, engine=’xlsxwriter’)

In [9]:
df1['correct_ratio1']=df1['correct_ratio'].astype(int)

In [10]:
df1[df1['correct_ratio1']>=90].shape,df1.shape

((255, 4), (322, 4))

In [13]:
df1.to_csv('data/matched_names.csv',index=False)